## Preparation

### Check assigned GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Mar 15 00:50:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Check assigned memory

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


### Download Object Detection API repo



In [ ]:
!rm -rf object-detection-api && git clone https://github.com/CatchZeng/object-detection-api.git

Cloning into 'object-detection-api'...
remote: Enumerating objects: 361, done.
remote: Counting objects: 100% (361/361), done.
remote: Compressing objects: 100% (237/237), done.
remote: Total 361 (delta 136), reused 317 (delta 92), pack-reused 0
Receiving objects: 100% (361/361), 1.77 MiB | 25.91 MiB/s, done.
Resolving deltas: 100% (136/136), done.


### Install Object Detection API

In [ ]:
!cd object-detection-api && make install

if [ -d "./models" ]; then \
    echo 'models downloaded'; \
  else \
        git clone --depth=1 https://github.com/tensorflow/models; \
fi
Cloning into 'models'...
remote: Enumerating objects: 3464, done.
remote: Counting objects: 100% (3464/3464), done.
remote: Compressing objects: 100% (2781/2781), done.
remote: Total 3464 (delta 1047), reused 1533 (delta 635), pack-reused 0
Receiving objects: 100% (3464/3464), 34.30 MiB | 31.53 MiB/s, done.
Resolving deltas: 100% (1047/1047), done.
cd models/research && \
protoc object_detection/protos/*.proto --python_out=. && \
cp object_detection/packages/tf2/setup.py . && \
python -m pip install --use-feature=2020-resolver .
Processing /content/object-detection-api/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.


### Go to workspace dir

In [ ]:
import os
os.chdir("/content/object-detection-api/workspace/test-mask")

### Download pre-trained Mask R-CNN model

In [ ]:
!make dl-model

mkdir -p pre-trained-models; \
model=mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8; \
curl -O http://download.tensorflow.org/models/object_detection/tf2/20200711/$model.tar.gz; \
tar zxvf $model.tar.gz; \
mv -f $model ./pre-trained-models/; \
rm -rf $model $model.tar.gz;
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  462M  100  462M    0     0   118M      0  0:00:03  0:00:03 --:--:--  118M
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0.data-00000-of-00001
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/checkpoint
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0.index
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/pipeline.config
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/

## Train model



In [ ]:
!make train

python model_main_tf2.py \
--model_dir=models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8 \
--pipeline_config_path=models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/pipeline.config
2022-03-15 00:53:29.008655: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0315 00:53:29.014656 140483420235648 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I0315 00:53:29.019054 140483420235648 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0315 00:53:29.019262 140483420235648 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename 

### FAQ

```
ImportError: cannot import name '_registerMatType' from 'cv2.cv2'
```

> https://stackoverflow.com/questions/70537488/cannot-import-name-registermattype-from-cv2-cv2

Execute the following scripts.

In [ ]:
!pip list | grep opencv-python-headless
!pip uninstall --yes opencv-python-headless
!pip install opencv-python-headless==4.1.2.30

opencv-python-headless        4.5.5.64
Found existing installation: opencv-python-headless 4.5.5.64
Uninstalling opencv-python-headless-4.5.5.64:
  Successfully uninstalled opencv-python-headless-4.5.5.64
     |████████████████████████████████| 21.8 MB 1.4 MB/s 


```
2 root error(s) found.
  (0) UNIMPLEMENTED:  DNN library is not found.
     [[{{node functional_1/conv1_conv/Conv2D}}]]
     [[StatefulPartitionedCall/SecondStagePostprocessor/BatchMultiClassNonMaxSuppression/MultiClassNonMaxSuppression/Reshape_5/_126]]
  (1) UNIMPLEMENTED:  DNN library is not found.
     [[{{node functional_1/conv1_conv/Conv2D}}]]
0 successful operations.
```
> https://stackoverflow.com/questions/71000120/colab-0-unimplemented-dnn-library-is-not-found

Execute the following scripts.

In [ ]:
!pip install --upgrade tf-models-official==2.8.0
!pip install --upgrade tensorflow==2.8.0

## Test model

In [10]:
!make export && python test_images.py

python exporter_main_v2.py \
--input_type image_tensor \
--pipeline_config_path models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/pipeline.config \
--trained_checkpoint_dir models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/ \
--output_directory exported-models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8
2022-03-15 01:50:51.997479: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
W0315 01:50:52.151746 140231422580608 deprecation.py:615] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:458: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with back_p

## Download models

In [11]:
from google.colab import files

!zip -r /content/exported-modles.zip /content/object-detection-api/workspace/test-mask/exported-models
files.download("/content/exported-modles.zip")

  adding: content/object-detection-api/workspace/test-mask/exported-models/ (stored 0%)
  adding: content/object-detection-api/workspace/test-mask/exported-models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/ (stored 0%)
  adding: content/object-detection-api/workspace/test-mask/exported-models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/pipeline.config (deflated 70%)
  adding: content/object-detection-api/workspace/test-mask/exported-models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ (stored 0%)
  adding: content/object-detection-api/workspace/test-mask/exported-models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0.data-00000-of-00001 (deflated 7%)
  adding: content/object-detection-api/workspace/test-mask/exported-models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0.index (deflated 81%)
  adding: content/object-detection-api/workspace/test-mask/exported-models/mask_rcnn_inception_resnet_v2_1024x102

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>